In [1]:

sealed trait Expr

case class Const(v: Double) extends Expr // Expr -> Const(v)
case class Ident(s: String) extends Expr // Expr -> Ident(s)

// Arithmetic Expressions
case class Plus(e1: Expr, e2: Expr) extends Expr // Expr -> Plus(Expr, E

//Let bindings
case class Let(s: String, defExpr: Expr, bodyExpr: Expr) extends Expr

//Function definition
case class FunDef(param: String, bodyExpr: Expr) extends Expr

// Function call
case class FunCall(funCalled: Expr, argExpr: Expr) extends Expr

// Let Var
case class LetVar(x: String, e1: Expr, e2: Expr) extends Expr

// Assign Var
case class AssignVar(x: String, e: Expr) extends Expr

defined trait Expr
defined class Const
defined class Ident
defined class Plus
defined class Let
defined class FunDef
defined class FunCall
defined class LetVar
defined class AssignVar

In [2]:
// Copy from the case for explicit references
sealed trait Value


/*-- Now we can finish the rest --*/
case class NumValue(f: Double) extends Value
case class BoolValue(b: Boolean) extends Value
/*-- Note: to get recursion working, we will need to make environments different --*/
case class Closure(x: String, e: Expr, pi: Map[String, Value]) extends Value 
/* -- references are here -- */
case class Reference(j: Int) extends Value
case object ErrorValue extends Value


/*2. Operators on values */

def valueToNumber(v: Value): Double = v match {
    case NumValue(d) => d
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a number")
}

def valueToBoolean(v: Value): Boolean = v match {
    case BoolValue(b) => b
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a boolean")
}

def valueToClosure(v: Value): Closure = v match {
    case Closure(x, e, pi) => Closure(x, e, pi)
    case _ =>  throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a closure")
}

    

defined trait Value
defined class NumValue
defined class BoolValue
defined class Closure
defined class Reference
defined object ErrorValue
defined function valueToNumber
defined function valueToBoolean
defined function valueToClosure

In [5]:
import scala.collection.mutable.ArrayBuffer

/*array buffer is mutable data structure
can grow and shrink 
it is like a dynamic tree, if you are about to fill up the buffer, it will double in size */

type Store= ArrayBuffer[Value]

def createNewCell (s: Store, v: Value): (Store, Int)= {
    //make a new cell
    val j=s.length
    val newstore= s:+(v)
    (newstore,j) //return is a store and address
}

//looking up value of address j
def lookupCellValue(s: Store, j: Int): Value = {
        if (j < s.length){
            s(j)
        } else {
            throw new IllegalArgumentException(s"Illegal lookup of nonexistant location $j") //aka out of bounds
        }
}
    
def assignToCell(s: Store, j: Int, v: Value): Store = {
        if (j< s.length){
            s(j)=v
        } else {
            throw new IllegalArgumentException(s"Illegal assignment to nonexistent location $j")
        }
    s //returning the origninal store
    }

import scala.collection.mutable.ArrayBuffer

/*array buffer is mutable data structure
can grow and shrink 
it is like a dynamic tree, if you are about to fill up the buffer, it will double in size */


defined type Store
defined function createNewCell
defined function lookupCellValue
defined function assignToCell

In [6]:
def evalExpr(e: Expr, env: Map[String, Value], store: Store): (Value, Store) = {
      /* Method to deal with binary arithmetic operations */
    
    def applyArith2 (e1: Expr, e2: Expr) (fun: (Double , Double) => Double) = {
        val (v1, store1) = evalExpr(e1, env, store) //first evaluate e1
        val (v2, store2) = evalExpr(e2, env, store1) //evaluate e2 under store from e1
        val v3 = fun(valueToNumber(v1), valueToNumber(v2)) //applying the funtion ex: v1+v2
        (NumValue(v3), store2) //returning the final value of v3 under the store2
    }  /* -- We have deliberately curried the method --*/
    
    /* Helper method to deal with unary arithmetic */
    def applyArith1(e: Expr) (fun: Double => Double) = {
        val (v,store1) = evalExpr(e, env, store)
        val v1 = fun(valueToNumber(v))
        (NumValue(v1), store1)
    }
    
    /* Helper method to deal with comparison operators */
    def applyComp(e1: Expr, e2: Expr) (fun: (Double, Double) => Boolean) = {
        val (v1, store1) = evalExpr(e1, env, store)
        val (v2, store2) = evalExpr(e2, env, store1)
        val v3 = fun(valueToNumber(v1), valueToNumber(v2))
        (BoolValue(v3), store2)
    }
    
    e match {
        case Const(f) => (NumValue(f), store) //same store unchanged 
        
        case Ident(x) => {
            if (env contains x ) { // In scala a.b(c) can simply be written as "a b c" <macro>
                val v = env(x)
                v match {
                    case Reference(j) => { // AUTO deref 
                        //if reference then you look it up in the store, get that value and return that
                        val v1 = lookupCellValue(store, j) // Lookup the store for address j
                        (v1, store) // return the value of reference(j) from the store.
                    }
                    case _ => (v, store) // return v and store unchanged
                    //if its not a reference, you just returns that value unchanged
                }  
            } else 
                throw new IllegalArgumentException(s"Undefined identifier $x")
        }
    
    
        case Plus(e1, e2) => applyArith2 (e1, e2) ( _ + _ )
            

        case Let(x, e1, e2) => {
            val (v1, store1) = evalExpr(e1, env, store)  // eval e1
            val env2 = env + (x -> v1) // create a new extended env
            evalExpr(e2, env2, store1) // eval e2 under that.
        }
    
        case FunDef(x, e) => {
            (Closure(x, e, env), store) // Return a closure with the current enviroment.
        }
        
        case FunCall(e1, e2) => {
            val (v1, store1) = evalExpr(e1, env, store)
            val (v2, store2) = evalExpr(e2, env, store1)
            v1 match {
                case Closure(x, closure_ex, closed_env) => {
                    // First extend closed_env by binding x to v2
                    val new_env = closed_env + ( x -> v2)
                    // Evaluate the body of the closure under the extended environment.
                    evalExpr(closure_ex, new_env, store2)
                }
                case _ => throw new IllegalArgumentException(s"Function call error: expression $e1 does not evaluate to a closure")
            }
        }
        
        
        
        case AssignVar(x, e) => { // x is a string -- name of identifier and e is Expr -- RHS of assignment
            val (v1, store1) = evalExpr(e, env, store) // First evaluate e
            val v2 =  if (env contains x)  // Next, check x from the current environment
                        env(x)
                      else 
                        throw new IllegalArgumentException(s"Undefined identifier $x")// Trying to assign to an undeclared identifier
            v2 match {
                case Reference(j) => { // x better be a reference in the current env.
                    val store3 = assignToCell(store1, j, v1) // assign to cell function in ImmutableStore API
                    (v1, store3) 
                }
                case _ => throw new IllegalArgumentException(s"AssignVar applied to argument that is not a mutable var")
                
            }
        }
        
        case LetVar(x, e1, e2) => { // let var x = e1 in e2 
            // This is the same treatment as let x = newref(e1) in e2 in ExplicitRef Language.
            val (v1, store1) = evalExpr(e1, env, store) // evaluate e1
            val (store2, j) = createNewCell(store1, v1) // create a new cell corresponding to the value of e1
            val newEnv = env + (x -> Reference(j)) // update the environment
            evalExpr(e2, newEnv, store2) // evaluatet e2 with the new environment and the new store.
        }
        
    }

}

    


defined function evalExpr

In [7]:
def evalTopLevel(e: Expr): Value= {
    // Start with empty environment and empty store
    val (v1, store) = evalExpr(e, Map(), new Store())
    println(s"Conclude Evaluation with store: $store")
    v1
}

defined function evalTopLevel